## Pipeline for microendoscopic data processing in CaImAn using the CNMF-E algorithm
This demo presents a complete pipeline for processing microendoscopic data using CaImAn. It includes:
- Motion Correction using the NoRMCorre algorithm
- Source extraction using the CNMF-E algorithm
- Deconvolution using the OASIS algorithm

Some basic visualization is also included. The demo illustrates how to `params`, `MoctionCorrection` and `cnmf` object for processing 1p microendoscopic data. For processing two-photon data consult the related `demo_pipeline.ipynb` demo. For more information see the companion CaImAn paper.

In [ ]:
try:
    get_ipython().run_line_magic('load_ext', 'autoreload')
    get_ipython().run_line_magic('autoreload', '2')
    get_ipython().run_line_magic('matplotlib', 'qt')
except:
    pass

import logging
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import scipy.io as sio
logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="caiman.log",
                    level=logging.DEBUG)

import caiman as cm
from caiman.source_extraction import cnmf
from caiman.utils.utils import download_demo
from caiman.utils.visualization import inspect_correlation_pnr, nb_inspect_correlation_pnr
from caiman.motion_correction import MotionCorrect
from caiman.source_extraction.cnmf import params as params
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour
import cv2

try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl
import holoviews as hv
bpl.output_notebook()
hv.notebook_extension('bokeh')

### Select file(s) to be processed
The `download_demo` function will download the specific file for you and return the complete path to the file which will be stored in your `caiman_data` directory. If you adapt this demo for your data make sure to pass the complete path to your file(s). Remember to pass the `fnames` variable as a list. Note that the memory requirement of the CNMF-E algorithm are much higher compared to the standard CNMF algorithm. Test the limits of your system before trying to process very large amounts of data.

In [ ]:
fnames = [r'G:\RL_CM2\analysis\SUPPORT.tif'.replace('\\','\\')]
print(fnames)

### Setup a cluster
To enable parallel processing a (local) cluster needs to be set up. This is done with a cell below. The variable `backend` determines the type of cluster used. The default value `'local'` uses the multiprocessing package. The `ipyparallel` option is also available. More information on these choices can be found [here](https://github.com/flatironinstitute/CaImAn/blob/master/CLUSTER.md). The resulting variable `dview` expresses the cluster option. If you use `dview=dview` in the downstream analysis then parallel processing will be used. If you use `dview=None` then no parallel processing will be employed.

In [ ]:
#%% start a cluster for parallel processing (if a cluster already exists it will be closed and a new session will be opened)
if 'dview' in locals():
    cm.stop_server(dview=dview)
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

### Setup some parameters
We first set some parameters related to the data and motion correction and create a `params` object. We'll modify this object with additional settings later on. You can also set all the parameters at once as demonstrated in the `demo_pipeline.ipynb` notebook.

In [ ]:
# dataset dependent parameters
frate = 10                      # movie frame rate
decay_time = 0.4                 # length of a typical transient in seconds

# motion correction parameters
motion_correct = True    # flag for performing motion correction
pw_rigid = True         # flag for performing piecewise-rigid motion correction (otherwise just rigid)
gSig_filt = (3, 3)       # size of high pass spatial filtering, used in 1p data
max_shifts = (20, 20)      # maximum allowed rigid shift
strides = (96, 96)       # start a new patch for pw-rigid motion correction every x pixels
overlaps = (32, 32)      # overlap between pathes (size of patch strides+overlaps)
max_deviation_rigid = 3  # maximum deviation allowed for patch with respect to rigid shifts
border_nan = 'copy'      # replicate values along the boundaries

num_frames_split = 50  #根据数据量大大小进行更改

mc_dict = {
    'fnames': fnames,
    'fr': frate,
    'decay_time': decay_time,
    'pw_rigid': pw_rigid,
    'max_shifts': max_shifts,
    'gSig_filt': gSig_filt,
    'strides': strides,
    'overlaps': overlaps,
    'max_deviation_rigid': max_deviation_rigid,
    'border_nan': border_nan,
    'num_frames_split':num_frames_split
}

opts = params.CNMFParams(params_dict=mc_dict)

### Motion Correction
The background signal in micro-endoscopic data is very strong and makes the motion correction challenging. 
As a first step the algorithm performs a high pass spatial filtering with a Gaussian kernel to remove the bulk of the background and enhance spatial landmarks. 
The size of the kernel is given from the parameter `gSig_filt`. If this is left to the default value of `None` then no spatial filtering is performed (default option, used in 2p data).
After spatial filtering, the NoRMCorre algorithm is used to determine the motion in each frame. The inferred motion is then applied to the *original* data so no information is lost.

The motion corrected files are saved in memory mapped format. If no motion correction is being performed, then the file gets directly memory mapped.

### Load memory mapped file

In [ ]:
fname_new=r'G:\RL_CM2\analysis\memmap_d1_1944_d2_1944_d3_1_order_C_frames_940.mmap'.replace('\\','\\');#如果第一次做MC就不需要运行这一行，如果做过MC，
                                                                                                   #不想再做一次，就可以直接读取MC后的mmap文件

In [ ]:
# load memory mappable file
Yr, dims, T = cm.load_memmap(fname_new)
images = Yr.T.reshape((T,) + dims, order='C')
print(images.shape)
print(Yr.shape)
print(dims)
print(T)

### Parameter setting for CNMF-E
We now define some parameters for the source extraction step using the CNMF-E algorithm. 
We construct a new dictionary and use this to modify the *existing* `params` object,

In [ ]:
# parameters for source extraction and deconvolution
p = 1               # order of the autoregressive system
K = None            # upper bound on number of components per patch, in general None
gSig = (2, 2)       # gaussian width of a 2D gaussian kernel, which approximates a neuron
gSiz = (4, 4)     # average diameter of a neuron, in general 4*gSig+1
Ain = None          # possibility to seed with predetermined binary masks
merge_thr = .7      # merging threshold, max correlation allowed
rf = 40             # half-size of the patches in pixels. e.g., if rf=40, patches are 80x80
stride_cnmf = 10    # amount of overlap between the patches in pixels
#                     (keep it at least large as gSiz, i.e 4 times the neuron size gSig)
tsub = 1            # downsampling factor in time for initialization,
#                     increase if you have memory problems
ssub = 1            # downsampling factor in space for initialization,
#                     increase if you have memory problems
#                     you can pass them here as boolean vectors
low_rank_background = None  # None leaves background of each patch intact,
#                     True performs global low-rank approximation if gnb>0
gnb = -2             # number of background components (rank) if positive,
#                     else exact ring model with following settings
#                         gnb= 0: Return background as b and W
#                         gnb=-1: Return full rank background B
#                         gnb<-1: Don't return background
nb_patch = 0        # number of background components (rank) per patch if gnb>0,
#                     else it is set automatically
min_corr = .65    # min peak value from correlation image
min_pnr = 35     # min peak to noise ration from PNR image
ssub_B = 2          # additional downsampling factor in space for background
ring_size_factor = 1.4  # radius of ring is gSiz*ring_size_factor

bord_px=0

merge_parallel=True


opts.change_params(params_dict={'method_init': 'corr_pnr',  # use this for 1 photon
                                'K': K,
                                'gSig': gSig,
                                'gSiz': gSiz,
                                'merge_thr': merge_thr,
                                'p': p,
                                'tsub': tsub,
                                'ssub': ssub,
                                'rf': rf,
                                'stride': stride_cnmf,
                                'only_init': True,    # set it to True to run CNMF-E
                                'nb': gnb,
                                'nb_patch': nb_patch,
                                'method_deconvolution': 'oasis',       # could use 'cvxpy' alternatively
                                'low_rank_background': low_rank_background,
                                'update_background_components': True,  # sometimes setting to False improve the results
                                'min_corr': min_corr,
                                'min_pnr': min_pnr,
                                'normalize_init': False,               # just leave as is
                                'center_psf': True,                    # leave as is for 1 photon
                                'ssub_B': ssub_B,
                                'ring_size_factor': ring_size_factor,
                                'del_duplicates': True, # whether to remove duplicates from initialization
                                'merge_parallel':merge_parallel,
                                'border_pix': bord_px})                # number of pixels to not consider in the borders)

### Inspect summary images and set parameters
Check the optimal values of `min_corr` and `min_pnr` by moving slider in the figure that pops up. You can modify them in the `params` object. 
Note that computing the correlation pnr image can be computationally and memory demanding for large datasets. In this case you can compute
only on a subset of the data (the results will not change). You can do that by changing `images[::1]` to `images[::5]` or something similar.
This will compute the correlation pnr image

In [ ]:
cn_filter, pnr = cm.summary_images.correlation_pnr_guohua(images[::3], gSig=2, swap_dim=False,center_psf=True) # change swap dim if output looks weird, it is a problem with tiffile
# inspect the summary images and set the parameters
nb_inspect_correlation_pnr(cn_filter.T, pnr.T)

In [ ]:
cn_filter, pnr = cm.summary_images.correlation_pnr_cuda(images[::1], gSig=2, swap_dim=False,center_psf=True) # change swap dim if output looks weird, it is a problem with tiffile
nb_inspect_correlation_pnr(cn_filter.T, pnr.T)

In [ ]:
v_search = cn_filter * pnr
v_search[(cn_filter < 0.1) | (pnr < 40)] = 0
img2save=v_search
min_value = np.nanmin(img2save)
img2save[np.isnan(img2save)]=min_value
print("minvalue:",min_value)
max_value = np.max(img2save)
print("maxvalue:",max_value)
min_value=0;
scaled_img2save = ((img2save - min_value) / (max_value - min_value) * 65535).astype(np.uint16)
cv2.imwrite('v_search2.png', scaled_img2save.T)

In [ ]:
cn, data_std,data_std2,data_max,data_filtered_mean1,data_filtered_mean2 = cm.summary_images.correlation_pnr_debug(images[::3], gSig=2, swap_dim=False,center_psf=True) 

In [ ]:
percentile_value = np.percentile(data_std2[data_std2 > 0], 0.5)
mask = (data_std2 > 0) & (data_std2 < percentile_value)
data_std2[mask] = percentile_value

In [ ]:
pnr = np.divide(data_max, data_std2)
pnr[pnr < 0] = 0
pnr2_1 = np.maximum(pnr, np.divide(data_std2,np.max(data_std2))*np.max(pnr))
pnr2_2 = pnr+np.divide(data_std2,np.max(data_std2))*np.max(pnr)
pnr2_2[pnr2_2 < 0] = 0
# pnr3 = np.maximum(pnr2, np.divide(data_std,np.max(data_std))*np.max(pnr2))
# pnr3 = np.maximum(pnr2, data_max)
# pnr3 = pnr2*data_max

In [ ]:
# import cv2
# import numpy as np
img2save=cn
min_value = np.nanmin(img2save)
img2save[np.isnan(img2save)]=min_value
print("minvalue:",min_value)
max_value = np.max(img2save)
print("maxvalue:",max_value)
min_value=0;
scaled_img2save = ((img2save - min_value) / (max_value - min_value) * 65535).astype(np.uint16)
cv2.imwrite('cn.png', scaled_img2save.T)

In [ ]:
data_std2.shape

You can inspect the correlation and PNR images to select the threshold values for `min_corr` and `min_pnr`. The algorithm will look for components only in places where these value are above the specified thresholds. You can adjust the dynamic range in the plots shown above by choosing the selection tool (third button from the left) and selecting the desired region in the histogram plots on the right of each panel.

In [ ]:
# # print parameters set above, modify them if necessary based on summary images
print(min_corr) # min correlation of peak (from correlation image)
print(min_pnr)  # min peak to noise ratio

### Run the CNMF-E algorithm

In [ ]:
cnm = cnmf.CNMF(n_processes=n_processes, dview=dview, Ain=Ain, params=opts)
# cnm.fit(images)

### Alternate way to run the pipeline at once
It is possible to run the combined steps of motion correction, memory mapping, and cnmf fitting in one step as shown below. The command is commented out since the analysis has already been performed. It is recommended that you familiriaze yourself with the various steps and the results of the various steps before using it.

## Component Evaluation

The processing in patches creates several spurious components. These are filtered out by evaluating each component using three different criteria:

- the shape of each component must be correlated with the data at the corresponding location within the FOV
- a minimum peak SNR is required over the length of a transient
- each shape passes a CNN based classifier

<img src="../../docs/img/evaluationcomponent.png"/>
After setting some parameters we again modify the existing `params` object.

In [ ]:
#%% COMPONENT EVALUATION
# the components are evaluated in three ways:
#   a) the shape of each component must be correlated with the data
#   b) a minimum peak SNR is required over the length of a transient
#   c) each shape passes a CNN based classifier
#Each parameter has a low threshold (rval_lowest (default -1), SNR_lowest (default 0.5), cnn_lowest (default 0.1)) and high threshold (rval_thr (default 0.8), min_SNR (default 2.5), min_cnn_thr (default 0.9)). A component has to exceed ALL low thresholds as well as ONE high threshold to be accepted.
#low threshold
rval_lowest =0.3
SNR_lowest =1
#high threshold
min_SNR = 3    # adaptive way to set threshold on the transient size
r_values_min = 0.7  # threshold on space consistency (if you lower more components
#                        will be accepted, potentially with worst quality)
cnm.params.set('quality', {'rval_lowest': rval_lowest,'SNR_lowest': SNR_lowest,'min_SNR': min_SNR,
                           'rval_thr': r_values_min,
                           'use_cnn': False})
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)

print(' ***** ')
print('Number of total components: ', len(cnm.estimates.C))
print('Number of accepted components: ', len(cnm.estimates.idx_components))

### Do some plotting

In [ ]:
import pickle
with open('cnm_estimates2.pkl', 'rb') as file:
    cnm.estimates = pickle.load(file)

In [ ]:
import pickle
with open('cnm_estimates2.pkl', 'wb') as file:
    pickle.dump(cnm.estimates, file)

In [ ]:
cn_pnr = cv2.imread('./pnr2_2_cn.png', cv2.IMREAD_ANYDEPTH).astype(np.float32)
percentile_value = np.percentile(cn_pnr[cn_pnr > 0], 99.9)
mask = (cn_pnr > 0) & (cn_pnr > percentile_value)
cn_pnr[mask] = percentile_value
cn_pnr.shape
print(percentile_value)

In [ ]:
#%% plot contour plots of accepted and rejected components
cnm.estimates.plot_contours_nb(img=(cn_pnr), idx=cnm.estimates.idx_components,thr=0.4)

In [ ]:
cnm.estimates.coordinates=None

In [ ]:
cnm.estimates.coordinates[1]

In [ ]:
good_coordinates=[cnm.estimates.coordinates[i] for i in cnm.estimates.idx_components]
ellipticity_threshhold=0.5
center = [np.clip(cor['CoM'], 0, 1944) for cor in good_coordinates if ((cor['ellipticity'] > ellipticity_threshhold) & (cor['ellipticity'] < 1.0/ellipticity_threshhold))]
center=np.array(center)
sio.savemat(fnames[0].split(".")[0]+'_center.mat',{"center":center})
# len(cc1)

In [ ]:
#把画出来good counter的坐标点导出来
good_coordinates=[cnm.estimates.coordinates[i] for i in cnm.estimates.idx_components]
ellipticity_threshhold=0.5
cc1 = [np.clip(cor['coordinates'][1:-1, 0], 0, 1944) for cor in good_coordinates if ((cor['ellipticity'] > ellipticity_threshhold) & (cor['ellipticity'] < 1.0/ellipticity_threshhold))]
cc2 = [np.clip(cor['coordinates'][1:-1, 1], 0, 1944) for cor in good_coordinates if ((cor['ellipticity'] > ellipticity_threshhold) & (cor['ellipticity'] < 1.0/ellipticity_threshhold))]
len(cc1)

## View traces of accepted and rejected components. Note that if you get data rate error you can start Jupyter notebooks using:
'jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10'

In [ ]:
# accepted components
cnm.estimates.hv_view_components(img=cn_pnr, idx=cnm.estimates.idx_components,
                                denoised_color='red', cmap='gray',coors=cnm.estimates.coordinates)

In [ ]:
# rejected components
cnm.estimates.hv_view_components(img=cn_pnr, idx=cnm.estimates.idx_components_bad,
                                denoised_color='red', cmap='gray',coors=cnm.estimates.coordinates)

### Save results to matlab mat

In [ ]:
sio.savemat(fnames[0].split(".")[0]+'_caiman_result.mat',{"A_neuron_good_idx":cnm.estimates.idx_components,"A_neuron_bad_idx":cnm.estimates.idx_components_bad,"C_trace":cnm.estimates.C,"A_neuron_sparse":cnm.estimates.A.astype(np.float64)})

In [ ]:
dir(cnm.estimates)

### Plot some figures

In [ ]:
C_trace = cnm.estimates.C
print("C_trace shape is：", C_trace.shape)
# imgplot = plt.imshow(C_trace[1:2000,:])
matplotlib.image.imsave(fnames[0].split(".")[0]+'C_trace.png', C_trace[cnm.estimates.idx_components[1:2000],:])

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as grid_spec

C_trace_good=C_trace[cnm.estimates.idx_components,:]
num2plot=150
gs = grid_spec.GridSpec(num2plot,1)
fig = plt.figure(figsize=(16,20))
fig.patch.set_facecolor('white')  # 设置整个图的背景颜色为黑色
ax_objs = []
for i in range(num2plot):
    trace=C_trace_good[i+1000,:]
    x_d = np.linspace(0,940, 940)

    # creating new axes object
    ax_objs.append(fig.add_subplot(gs[i:i+1, 0:]))

    # plotting the distribution
    ax_objs[-1].plot(x_d, trace,color="blue",lw=1)
    # ax_objs[-1].fill_between(x_d, np.exp(logprob), alpha=1,color=colors[i])


    # setting uniform x and y lims
    ax_objs[-1].set_xlim(0,940)
    # ax_objs[-1].set_ylim(0,2.5)

    # make background transparent
    rect = ax_objs[-1].patch
    rect.set_alpha(0)
    ax_objs[-1].set_axis_off()

gs.update(hspace=-0.7)

# fig.text(0.07,0.85,"Distribution of Aptitude Test Results from 18 – 24 year-olds",fontsize=20)

plt.tight_layout()
plt.show()
plt.savefig(fnames[0].split(".")[0]+'100_trace.png', dpi=500)

In [ ]:
x_d.shape

### Stop cluster

In [ ]:
cm.stop_server(dview=dview)

### Some instructive movies
Play the reconstructed movie alongside the original movie and the (amplified) residual

In [ ]:
#  #with background 
# cnm.estimates.play_movie(images, q_max=99.5, magnification=0.5,frame_range=slice(1, 1000, 5),display=True,
#                                   include_bck=True, save_movie=True, movie_name='/home/chang/caiman_data/example_movies/Caiman_1/Video1-14_1-5-1000.avi',use_color=True, gain_res=10, bpx=bord_px)

In [ ]:
# without background
cnm.estimates.play_movie(images, q_max=99.9, q_min=1,magnification=1,frame_range=slice(1, 100, 1),display=True,
                                 include_bck=False, save_movie=True, movie_name='F:\\Dr_Miniscope\\NO\\No\\2023_06_09\\1\\Miniscope\\without_background.mp4',use_color=True, gain_res=3, bpx=bord_px)

In [ ]:
#type(images)

In [ ]:
##--------------Save mmap file into tif file


#cm.load(‘filename.mmap’).save(‘filename.tif’)
#x1 = np.memmap(folder+'msCam1_rig__d1_480_d2_752_d3_1_order_F_frames_1000_.mmap', mode='r', shape=(1000,480,752),order='F')
#m_els = cm.load('/home/generalubuntu/mnt/Muad/ca-imaging/scalpel-test/registered_els__d1_346_d2_464_d3_1_order_F_frames_12600_.mmap', in_memory=True)

##--------------Save mmap file into tif file

## Save raw traces

Save the raw fluorescence trace of each cell by adding the denoised calcium trace `cnm.estimates.C` and the residual `cnm.estimates.YrA`. The `cnm.estimates.idx_components` is a list of cells that passed the `min_SNR` and `r_values_min` cutoffs from the **"Component Evaluation"** step. 

We're saving the whole `cnm.estimates` object for future use as well. If you want to know what this object contains, refer to CaImAn [estimates.py](https://github.com/flatironinstitute/CaImAn/blob/master/caiman/source_extraction/cnmf/estimates.py). Make sure to change both paths to where you placed **"Analysis_Pipeline-OASIS.ipynb"**.

In [ ]:
import pickle
fname = 'E:/CNT 4254 soc vs obj_NAc/estimates.obj'

with open(fname, 'wb') as file:
    pickle.dump(cnm.estimates, file)


good_caAct = cnm.estimates.C[cnm.estimates.idx_components, :]
good_resid = cnm.estimates.YrA[cnm.estimates.idx_components, :]

good_rawFl = good_caAct + good_resid

fname2 = 'E:/CNT 4254 soc vs obj_NAc/raw_fluorescence.obj'

with open(fname2, 'wb') as file:
    pickle.dump(good_rawFl, file)

In [ ]:
good_rawFl

In [ ]:
import scipy.io as sio
sio.savemat('E:/CNT 4254 soc vs obj_NAc/C_Raw.mat', {"C_Raw":good_rawFl})